In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, roc_auc_score
import os
import gc

In [3]:
# Set parameters

from pylab import rcParams
rcParams['figure.figsize'] = 15, 10
rcParams['font.size'] = 20
rcParams['axes.facecolor'] = 'white'
plots_rgb_blue = 'rgb(31,119,180)'
%matplotlib inline

## Process geographical data

Crashes in the Crash Analysis System (CAS) are geolocated using the NZ system coordinates (columns X and Y in the dataset) and by a mesh block. Meshblocks are small geographical units that vary in size and cover 30-60 dwellings. These geographical units are too small for our application, and we opted to use the larger geographical units, Statistical Area 2 (SA2) or Statistical Area (SA3) units. We map the meshblocks from the CAS to the relevant SA2 and SA3 units using the Meshblock Higher Geographies 2024 dataset from data.govt.nz (https://catalogue.data.govt.nz/dataset/meshblock-higher-geographies-2024). We find cases without mapping between the meshblock and the statistical areas dataset. We train a k-Neighbours model to predict the correct mapping of un-mapped meshblocks using their distance to mapped meshblocks using the coordinates of crashes within the meshblocks.

In [3]:
# Load the Auckland area meshblocks mapping data
meshblock_auckland_full_df = pd.read_csv('meshblock-higher-geographies-2024-auckland.csv')
meshblock_auckland_full_df.head()

,WKT,MB2024_V1_00,SA12023_V1_00,SA22023_V1_00,SA22023_V1_00_NAME,SA22023_V1_00_NAME_ASCII,SA32023_V1_00,SA32023_V1_00_NAME,SA32023_V1_00_NAME_ASCII,UR2023_V1_00,...,FUA2023_V1_00_NAME_ASCII,IFUA2023_V1_00,IFUA2023_V1_00_NAME,TFUA2023_V1_00,TFUA2023_V1_00_NAME,LANDWATER,LANDWATER_NAME,LAND_AREA_SQ_KM,AREA_SQ_KM,Shape_Length
0,"POLYGON ((1753128.8397 5915046.9774,1753132.89...",4019145,7037220,135500,Ōwairaka West,Owairaka West,51440,Mount Albert,Mount Albert,1108,...,Auckland,101,Urban core,1,Metropolitan area,12,Mainland,0.053445,0.053445,1183.791512
1,"POLYGON ((1758471.9046 5920096.1957,1758519.93...",4001328,7037219,136400,Parnell West,Parnell West,51200,Parnell,Parnell,1108,...,Auckland,101,Urban core,1,Metropolitan area,12,Mainland,0.009572,0.009572,421.238251
2,"POLYGON ((1772737.6177 5879642.14,1772741.0269...",827902,7032417,169901,Tuakau Rural,Tuakau Rural,52450,Tuakau,Tuakau,1164,...,Auckland,201,Hinterland,1,Metropolitan area,12,Mainland,1.450875,1.450875,5144.014389
3,"POLYGON ((1772799.1844 5892796.4598,1772800.50...",4014920,7032175,164302,Drury East,Drury East,52250,Drury,Drury,1108,...,Auckland,101,Urban core,1,Metropolitan area,12,Mainland,0.261285,0.261285,3082.177087
4,"POLYGON ((1773448.2967 5893337.3844,1773557.31...",4004236,7010152,164302,Drury East,Drury East,52250,Drury,Drury,1108,...,Auckland,101,Urban core,1,Metropolitan area,12,Mainland,0.272358,0.272358,2963.151998


In [4]:
meshblock_auckland_df = meshblock_auckland_full_df[['MB2024_V1_00', 'SA22023_V1_00','SA22023_V1_00_NAME_ASCII']]

In [13]:
# Load the Crash Analysis System (CAS) data set and add the additional data on the mesh blocks
cas_df = pd.read_csv('Crash_Analysis_System_(CAS)_data.csv')
cas_mesh_df = pd.merge(cas_df,meshblock_auckland_df, left_on='meshblockId', right_on='MB2024_V1_00', how='left')
number_of_crashes_rows = cas_mesh_df.shape[0]
number_missing_SA2 = cas_mesh_df['SA22023_V1_00'].isna().sum()
print(f'Out of {number_of_crashes_rows} rows in the crashes dataset there are {number_missing_SA2} rows were meshblock are not mapped to SA2')

Out of 285346 rows in the crashes dataset there are 99106 rows were meshblock are not mapped to SA2


In [14]:
# We want to check the percentage of NaN values in SA22023_V1_00 in each crash year

filtered_df = cas_mesh_df[['OBJECTID','SA22023_V1_00','crashFinancialYear']]
nan_percentage = filtered_df.groupby('crashFinancialYear')['SA22023_V1_00'].apply(lambda x: x.isna().mean() * 100).reset_index(name='NaN_Percentage_SA22023_V1_00')

fig = px.line(nan_percentage, x='crashFinancialYear', y='NaN_Percentage_SA22023_V1_00', 
              title='Percentage of NaN Values in SA22023_V1_00 by Crash Financial Year', 
              labels={'crashFinancialYear': 'Crash Financial Year', 'NaN_Percentage_SA22023_V1_00': 'Percentage of NaN Values'})
fig.update_layout(yaxis=dict(range=[0, nan_percentage['NaN_Percentage_SA22023_V1_00'].max() + 5]))
fig.show()

The percentage of missing values is stable across the years.

In [19]:
# Check how many meshbleocks do not have mapping to SA2
filtered_block_df = cas_mesh_df[['OBJECTID','SA22023_V1_00','meshblockId']]
block_nan_percentage = filtered_block_df.groupby('meshblockId')['SA22023_V1_00'].apply(lambda x: x.isna().mean() * 100).reset_index(name='NaN_Percentage_SA22023_V1_00')
block_nan_percentage['NaN_Percentage_SA22023_V1_00'].value_counts()

NaN_Percentage_SA22023_V1_00
0.0      8344
100.0    1885
Name: count, dtype: int64

There are 8,344 meshblocks mapped to SA2 and 1,885 which are not mapped.

### Find missing mapping using a k-Neighbours model

We use the NZ coordinates (X and Y columns) crashes in mapped meshblocks to predict the mapping for unmapped meshblocks.

In [18]:
# Split the data into training (non-missing) and prediction (missing) sets
train_df = cas_mesh_df[cas_mesh_df['SA22023_V1_00'].notna()]
predict_df = cas_mesh_df[cas_mesh_df['SA22023_V1_00'].isna()]

# Create features and target variables for training
X = train_df[['X', 'Y']]
y = train_df['SA22023_V1_00']

# Split the training data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

kNN with uniform weightings and k neigbours between 1 and 15

In [21]:
k=15
training_error = pd.Series(index=range(1, k+1), dtype=float)
test_error = pd.Series(index=range(1, k+1), dtype=float)
for i in range(1, k+1):
    knn = KNeighborsClassifier(n_neighbors=i, weights='uniform')
    knn.fit(X_train, y_train)
    training_error[i] = 1.0 - knn.score(X_train, y_train)
    test_error[i] = 1.0 - knn.score(X_test, y_test)

# Plot the training and test error for each value of k
trace1 = go.Scatter(x=training_error.index, y=training_error, mode='lines+markers', name='Training Error')
trace2 = go.Scatter(x=test_error.index, y=test_error, mode='lines+markers', name='Test Error')
fig = go.Figure(data=[trace1, trace2])
fig.update_layout(
    title='Generalisation, Convergence and Overfitting - uniform',
    xaxis_title='Classifier Complexity (k)',
    yaxis_title='Error Rate / Misclassification Rate',
    xaxis=dict(range=[1, k], tickmode='linear'),
    yaxis=dict(range=[0, 1])
)
fig.show()

kNN with distance weightings and k neigbours between 1 and 15

In [22]:
k=15
training_error = pd.Series(index=range(1, k+1), dtype=float)
test_error = pd.Series(index=range(1, k+1), dtype=float)
for i in range(1, k+1):
    knn = KNeighborsClassifier(n_neighbors=i, weights='distance')
    knn.fit(X_train, y_train)
    training_error[i] = 1.0 - knn.score(X_train, y_train)
    test_error[i] = 1.0 - knn.score(X_test, y_test)
    
# Plot the training and test error for each value of k
trace1 = go.Scatter(x=training_error.index, y=training_error, mode='lines+markers', name='Training Error')
trace2 = go.Scatter(x=test_error.index, y=test_error, mode='lines+markers', name='Test Error')
fig = go.Figure(data=[trace1, trace2])
fig.update_layout(
    title='Generalisation, Convergence and Overfitting - distance',
    xaxis_title='Classifier Complexity (k)',
    yaxis_title='Error Rate / Misclassification Rate',
    xaxis=dict(range=[1, k], tickmode='linear'),
    yaxis=dict(range=[0, 1])
)
fig.show()

Train a kNN classifier with distance weightings and k neighbours = 5

In [23]:
# Initialize the kNN classifier
knn = KNeighborsClassifier(n_neighbors=10 ,weights='distance')
# Train the classifier on the full training set
knn.fit(X_train, y_train)

# Evaluate the classifier on the test set
test_score = knn.score(X_test, y_test)
print(f"Test set score: {test_score}")

# Predict on the test set
y_pred = knn.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred, zero_division=0))

Test set score: 0.9580380154639175
              precision    recall  f1-score   support

    110700.0       1.00      1.00      1.00        45
    110800.0       1.00      1.00      1.00        40
    110900.0       0.94      0.95      0.95        66
    111100.0       0.88      1.00      0.94        22
    111200.0       0.99      0.97      0.98        99
    111300.0       0.96      1.00      0.98        26
    111400.0       1.00      0.85      0.92        33
    111601.0       1.00      0.90      0.95        21
    111602.0       0.95      1.00      0.98        20
    111700.0       0.93      1.00      0.97        14
    111901.0       1.00      1.00      1.00         1
    112101.0       0.94      0.96      0.95        67
    112200.0       1.00      1.00      1.00         2
    112301.0       0.93      0.88      0.90        32
    112401.0       1.00      1.00      1.00        47
    112500.0       0.86      0.86      0.86        28
    112600.0       0.89      0.98      0.94   

In [24]:
# Predict the missing values using the kNN model
# Features for prediction
X_predict = predict_df[['X', 'Y']]

# Predict the missing values
predictions = knn.predict(X_predict)

# Create a new column and fill the missing values with the predicted ones
cas_mesh_df['SA22023_V1_00_Filled'] = cas_mesh_df['SA22023_V1_00']

# Fill the missing values in the original DataFrame
cas_mesh_df.loc[cas_mesh_df['SA22023_V1_00_Filled'].isna(), 'SA22023_V1_00_Filled'] = predictions
cas_mesh_df['SA22023_V1_00_Filled'] =  cas_mesh_df['SA22023_V1_00_Filled'].astype('int64')

### Prepare the dataset

In [23]:
# Preapre additional information at the SA2 level
meshblock_auckland_SA2_level_df = meshblock_auckland_full_df.iloc[:,3:9]
meshblock_auckland_SA2_level_df = meshblock_auckland_SA2_level_df.drop_duplicates()
meshblock_auckland_SA2_level_df.head()

,SA22023_V1_00,SA22023_V1_00_NAME,SA22023_V1_00_NAME_ASCII,SA32023_V1_00,SA32023_V1_00_NAME,SA32023_V1_00_NAME_ASCII
0,135500,Ōwairaka West,Owairaka West,51440,Mount Albert,Mount Albert
1,136400,Parnell West,Parnell West,51200,Parnell,Parnell
2,169901,Tuakau Rural,Tuakau Rural,52450,Tuakau,Tuakau
3,164302,Drury East,Drury East,52250,Drury,Drury
5,152300,East Tamaki,East Tamaki,51820,East Tamaki,East Tamaki


In [56]:
# Add the additional information to the meshblock data set and save to a CSV file
cas_mesh_merged_df = pd.merge(cas_mesh_df,meshblock_auckland_SA2_level_df, left_on='SA22023_V1_00_Filled', right_on='SA22023_V1_00', how='left')
cas_mesh_merged_df = cas_mesh_merged_df.drop(columns=['SA22023_V1_00_NAME_ASCII_x','SA22023_V1_00_y'])
cas_mesh_merged_df = cas_mesh_merged_df.rename(columns={'SA22023_V1_00_x' : 'SA22023_V1_00_original', 'SA22023_V1_00_NAME_ASCII_y':'SA22023_V1_00_NAME_ASCII'})
cas_mesh_merged_df.to_csv('cas_merged_with_SA2_data.csv', index=False)

## Process auxilary data

### Daily highway traffic counts data

We use highway traffic count to represent the traffic volumes in Auckland - https://opendata-nzta.opendata.arcgis.com/datasets/NZTA::tms-daily-traffic-counts-api/explore

In [4]:
daily_traffic_volume = pd.read_csv('Auckland_TMS_Telemetry_Sites.csv')

# Split the Start Date and check if the time provided was in the am or pm. If it was pm then add one day to the reported data
daily_traffic_volume['Split Start Date'] = daily_traffic_volume['Start Date'].str.split()
daily_traffic_volume['Date'] = daily_traffic_volume['Split Start Date'].str[0]
daily_traffic_volume['AM PM'] = daily_traffic_volume['Split Start Date'].str[2]
daily_traffic_volume['Date'] = pd.to_datetime(daily_traffic_volume['Date'])
daily_traffic_volume['Adjusted Date'] = np.where(daily_traffic_volume['AM PM'] == 'PM', daily_traffic_volume['Date'] + pd.Timedelta(days=1), daily_traffic_volume['Date'])


In [5]:
# Create additional columns from the "Start Date" column
daily_traffic_volume['Period'] = daily_traffic_volume['Date'].apply(lambda x: x.replace(day=1))
daily_traffic_volume['Year'] = daily_traffic_volume['Date'].dt.year
daily_traffic_volume['Month'] = daily_traffic_volume['Date'].dt.month
daily_traffic_volume['HalfYear'] = daily_traffic_volume.apply(lambda row: f"{row['Year']}H1" if row['Month'] <= 6 else f"{row['Year']}H2", axis=1)
filtered_daily_traffic_volume =  daily_traffic_volume[(daily_traffic_volume['Date'] >= '2018-01-01')&(daily_traffic_volume['Date'] <= '2024-04-30')]

#### Daily traffic volumes

In [6]:

agg_daily_traffic_volume = filtered_daily_traffic_volume.groupby(['Year',
                                                                  'Date','HalfYear','Period'])['Traffic Count'].sum().reset_index()
fig = px.line(agg_daily_traffic_volume, x='Date', y='Traffic Count', 
              title='Traffic Count Over Time')
fig.update_yaxes(range=[0, agg_daily_traffic_volume['Traffic Count'].max()])
fig.show()

We note a significant drop in traffic during the first COVID-19 lockdown in March and April 2020 and during the second Auckland lockdown in August and September 2021.

#### Aggregate to half year level

The crashes dataset doesn't reveal the exact dates of the accidents, but only the year and financial year in which each accident happened. From this information, we will later find in which half of each year each accident happened. To join the traffic information to the crashes dataset, we aggregate the traffic volume to half annual level.

In [7]:
# Total traffic for each half year
total_half_year_traffic = agg_daily_traffic_volume.groupby('HalfYear')['Traffic Count'].sum().reset_index()
fig = px.bar(total_half_year_traffic, x='HalfYear', y='Traffic Count', 
              title='Total Traffic Per Half Year')
fig.update_layout(
    xaxis=dict(
        tickmode='linear',
        tick0=min(total_half_year_traffic['HalfYear']),
        dtick=1,
        tickangle=90
    )
)
fig.show()

In [8]:
# Average daily traffic for each half year
average_daily_traffic_half_year = agg_daily_traffic_volume.groupby('HalfYear')['Traffic Count'].mean().reset_index()
fig = px.bar(average_daily_traffic_half_year, x='HalfYear', y='Traffic Count', 
              title='Average Daily Traffic - Per Half Year')
fig.update_layout(
    xaxis=dict(
        tickmode='linear',
        tick0=min(average_daily_traffic_half_year['HalfYear']),
        dtick=1,
        tickangle=90
    )
)
fig.show()

In [9]:
# Save the daily traffic to a csv file
average_daily_traffic_half_year.to_csv('auckland_daily_average_traffic_half_year.csv',index=False)

### Population data

We use estimated pupulation figures for the Auckland metro area sourced from https://www.macrotrends.net/global-metrics/cities/21957/auckland/population.
The pouplation is estimated as at the end of each year.

In [11]:
auckland_population_df = pd.read_csv('Auckland-population.csv', skiprows=14)
auckland_population_df['Year']  = auckland_population_df['date'].str[:4].astype(int)
auckland_population_df = auckland_population_df.drop(columns=([' Annual Change']))
auckland_population_df =  auckland_population_df.rename(columns={' Population': 'Population'})
auckland_population_df =  auckland_population_df[(auckland_population_df['Year'] >=1999) & (auckland_population_df['Year'] <=2026)]

fig = px.bar(auckland_population_df, x='Year', y='Population', 
              title='Estimated Auckland Metro Area Population By Year')
fig.update_layout(
    xaxis=dict(
        tickmode='linear',
        tick0=min(auckland_population_df['Year']),
        dtick=1,
        tickangle=90
    )
)
fig.show()

To align the estimated population figures with the crash dataset, we extrapolate the population projections to the middle of each half-year.

In [13]:
# Estimate the population in the middle of each half year

new_rows = []

for i in range(1, len(auckland_population_df)):
    prev_year_pop = auckland_population_df['Population'].iloc[i-1]
    curr_year_pop = auckland_population_df['Population'].iloc[i]
    year_diff = curr_year_pop - prev_year_pop

    # Estimate for March (end of Q1)
    march_pop = prev_year_pop + (year_diff / 4)
    new_rows.append({'HalfYear': f"{auckland_population_df['Year'].iloc[i]}H1", 'Population': march_pop})

    # Estimate for September (end of Q3)
    september_pop = prev_year_pop + (3 * year_diff / 4)
    new_rows.append({'HalfYear': f"{auckland_population_df['Year'].iloc[i]}H2", 'Population': september_pop})

auckland_half_year_population = pd.DataFrame(new_rows)

fig = px.bar(auckland_half_year_population, x='HalfYear', y='Population', 
              title='Estimated Auckland Metro Area Population By Half Year')
fig.update_layout(
    xaxis=dict(
        tickmode='linear',
        tick0=min(auckland_half_year_population['HalfYear']),
        dtick=1,
        tickangle=90
    )
)
fig.show()

In [ ]:
# Save the daily traffic to a csv file
auckland_half_year_population.to_csv('auckland_half_annual_population.csv', index=False)